In [1]:
import numpy as np

import matplotlib.pyplot as plt
import pandas as pd

from classevy.klas import StudentGroup, PlanPopulation, Plan

In [2]:
students = StudentGroup('../data/students.csv')

/Users/mtytgat/code/classevy/src/classevy/klas.py:133: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self.required_columns = required_columns


In [3]:
students

,name,score_math,score_spelling,gender,learning_advantage,not_together,together,preferences
number,,,,,,,,
1,luuk,10,8,1,1,"(4,)","(2,)","(3, 6, 17)"
2,pat,9,7,1,0,(),"(1,)","(5, 9, 27)"
3,jos,8,6,1,0,"(6,)",(),"(2, 1, 21)"
4,pol,7,9,1,0,"(1,)",(),"(5, 6, 7)"
5,katrien,6,7,-1,0,(),(),"(6, 7, 23)"
6,mo,5,5,1,0,"(3, 8)",(),"(12, 5, 25)"
7,tia,6,7,-1,0,(),"(8,)","(8, 17, 23)"
8,karl,7,4,1,0,"(6,)","(7,)","(9, 27, 14)"
9,siem,8,3,1,0,(),(),"(10, 18, 19)"


In [4]:
plan = Plan(students,2)

/Users/mtytgat/code/classevy/src/classevy/klas.py:133: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self.required_columns = required_columns


In [34]:
sum(students.loc[list(curr_pref), "gender"] == 1)

3

In [35]:
len(students)

27

In [36]:
plan.do_assignment(flag_prio_prefs=True)

NameError: name 'plan' is not defined

In [37]:
plan.students

NameError: name 'plan' is not defined

In [9]:
plan.improve_preferences()
plan.students

,name,score,gender,age,not_together,together,preferences,options,dna_assignment,final_assignment,pref_satisfied
number,,,,,,,,,,,
1,luuk,10,1,6,"(4,)","(2,)","(3, 6, 17)",[1],1,1.0,2.0
2,pat,9,1,7,(),"(1,)","(5, 9, 27)",[1],0,1.0,1.0
3,jos,8,1,6,"(6,)",(),"(2, 1, 21)",[1],0,1.0,2.0
4,pol,7,1,7,"(1,)",(),"(5, 6, 7)",[0],1,0.0,3.0
5,katrien,6,-1,7,(),(),"(6, 7, 23)","[0, 1]",0,0.0,3.0
6,mo,5,1,6,"(3, 8)",(),"(12, 5, 25)",[],1,0.0,3.0
7,tia,6,-1,7,(),"(8,)","(8, 17, 23)",[0.0],0,0.0,2.0
8,karl,7,1,6,"(6,)","(7,)","(9, 27, 14)",[0],1,0.0,1.0
9,siem,8,1,6,(),(),"(10, 18, 19)","[0, 1]",1,0.0,1.0


In [10]:
plan.assignment_check

True

In [11]:
k0 = plan.classes[0]
k1 = plan.classes[1]

In [12]:
k0.print_names()
k1.print_names()

pol
katrien
mo
tia
karl
siem
ahmed
pius
darius
marius
maria
maarten
sanne
rinus
carla
clara
luuk
pat
jos
mario
marie
peter
pieter
pier
pieter
karla
klara


In [13]:
k0.students

,name,score,gender,age,not_together,together,preferences,options,dna_assignment,final_assignment,pref_satisfied
number,,,,,,,,,,,
4,pol,7,1,7,"(1,)",(),"(5, 6, 7)",[0],1,0.0,3.0
5,katrien,6,-1,7,(),(),"(6, 7, 23)","[0, 1]",0,0.0,3.0
6,mo,5,1,6,"(3, 8)",(),"(12, 5, 25)",[],1,0.0,3.0
7,tia,6,-1,7,(),"(8,)","(8, 17, 23)",[0.0],0,0.0,2.0
8,karl,7,1,6,"(6,)","(7,)","(9, 27, 14)",[0],1,0.0,1.0
9,siem,8,1,6,(),(),"(10, 18, 19)","[0, 1]",1,0.0,1.0
10,ahmed,8,1,6,(),"(11,)","(12, 5, 25)",[0.0],1,0.0,3.0
11,pius,5,1,7,(),"(10,)","(2, 4, 6)",[0.0],1,0.0,2.0
12,darius,4,1,7,"(20,)",(),"(10, 18, 19)",[0],1,0.0,1.0


In [14]:
k1.students

,name,score,gender,age,not_together,together,preferences,options,dna_assignment,final_assignment,pref_satisfied
number,,,,,,,,,,,
1,luuk,10,1,6,"(4,)","(2,)","(3, 6, 17)",[1],1,1.0,2.0
2,pat,9,1,7,(),"(1,)","(5, 9, 27)",[1],0,1.0,1.0
3,jos,8,1,6,"(6,)",(),"(2, 1, 21)",[1],0,1.0,2.0
14,mario,9,1,8,(),(),"(1, 3, 5)","[0, 1]",1,1.0,2.0
16,marie,8,-1,7,(),(),"(3, 6, 17)","[0, 1]",1,1.0,2.0
17,peter,9,1,7,(),(),"(9, 13, 14)","[0, 1]",1,1.0,1.0
18,pieter,10,1,7,(),(),"(2, 1, 21)","[0, 1]",1,1.0,2.0
19,pier,6,1,7,(),(),"(7, 19, 17)","[0, 1]",1,1.0,2.0
20,pieter,8,1,7,"(12,)",(),"(3, 22, 1)",[1],0,1.0,2.0


In [15]:
len(k0.students)

16

In [16]:
plan.students.properties

['score',
 'gender',
 'age',
 'options',
 'dna_assignment',
 'final_assignment',
 'pref_satisfied']

In [19]:
print('Spread of mean(age) between classes', plan.spread_age)
print('Spread of mean(gender) between classes', plan.spread_gender)
print('Spread of mean(score) between classes', plan.spread_score)
print('Spread of size between classes', plan.spread_size)

Spread of mean(age) between classes 0.14204545454545459
Spread of mean(gender) between classes 0.10227272727272727
Spread of mean(score) between classes 0.84375
Spread of size between classes 2.5


In [20]:
print('mean(age) of classes', plan.classes_age)
print('mean(gender) of classes', plan.classes_gender)
print('mean(score) of classes', plan.classes_score)
print('size of classes', plan.classes_size)

mean(age) of classes [6.625, 6.909090909090909]
mean(gender) of classes [0.25, 0.45454545454545453]
mean(score) of classes [6.3125, 8.0]
size of classes [16, 11]


In [25]:
def make_pol():
    global pol
    pol = 1

In [26]:
make_pol()

In [27]:
pol

1

In [28]:
students.index.values

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27])

In [43]:
plan.final_assignment

array([1., 1., 1., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1.,
       1., 1., 1., 1., 1., 0., 1., 0., 0., 1.])

In [47]:
pref = students['preferences'].loc[1]

In [52]:
%%time
students.loc[list(pref)]['gender']

CPU times: user 1 ms, sys: 62 µs, total: 1.06 ms
Wall time: 1.05 ms


number
3     1
6     1
17    1
Name: gender, dtype: int64

In [55]:
%%time
students.at[list(pref),'gender']

InvalidIndexError: [3, 6, 17]

In [54]:
%%time
students['gender'].loc[list(pref)]

CPU times: user 980 µs, sys: 77 µs, total: 1.06 ms
Wall time: 1.05 ms


number
3     1
6     1
17    1
Name: gender, dtype: int64

In [58]:
[i for i, stu in students.iterrows()]

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27]

In [6]:
plan.write_excel('temp.xlsx')

/Users/mtytgat/code/classevy/src/classevy/klas.py:133: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self.required_columns = required_columns
/Users/mtytgat/code/classevy/src/classevy/klas.py:133: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self.required_columns = required_columns
/Users/mtytgat/code/classevy/src/classevy/klas.py:133: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self.required_columns = required_columns
/Users/mtytgat/code/classevy/src/classevy/klas.py:133: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access